In [ ]:
import json

with open("essential_account.json","r",encoding="UTF-8") as file:
    data = json.load(file)
    accounts = data["essential_accounts"]

for account in accounts:
    print(account)
    account["name"]




In [ ]:
import pandas as pd 

from player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager
product_A = game_master.construct_instance("inventory","product_A").get("ID")
bilding_B = game_master.construct_instance("building", "building_B", address="東京都品川区荏原3-7-13").get("ID")
asset_list = list(game_master.asset_registry.keys())
player1.redister_product(product_A)
player1.aquire_building(bilding_B, 2000)

player1.purchase_product(product_A, 100, 150)
player1.purchase_product(product_A, 200, 120, 15)

player1.sale_product(product_A, 80)

player1.purchase_product(product_A, 30, 160)
player1.sale_product(product_A, 100, 225)

player1.sale_product(product_A, 100, 210, 500)

player1.perform_inventory_audit(product_A)

end_1 = ledger1.execute_settlement()

ledger1.display_transaction_history()
# ledger1.display_trial_balance(end_1)
# ledger1.display_financial_statements(end_1)

df = pd.DataFrame.from_dict(end_1, orient="index")


In [ ]:
ledger1.display_financial_statements(end_1)

In [ ]:
inventory_c = game_master.construct_instance("inventory", "inventory_c")
type(inventory_c.get("class"))


In [ ]:
import pandas as pd 

from player import *

game_master = GameMaster()
player1 = Player("player1",game_master)
ledger1 = player1.ledger_manager

building_B = game_master.construct_instance("building")

In [ ]:
from player import *
 # ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーを作成して登録
player = Player(name="Player1", game_master=game_master)
game_master.players.append(player)

# 資産の生成と登録
tangible_asset = game_master.construct_instance(
    asset_type="building",
    name="OfficeBuilding",
    value=1000000,
    address="Central Tokyo"
)["ID"]

# プレイヤーに資産を取得させる
player.aquire_building(asset_id=tangible_asset, value=1000000)

# ゲーム内時間を90日進める
print("\n=== 時間を90日進めます ===")
game_master.advance_time(days=90)

# 資産の状態を表示
print("\n=== 資産の状態 ===")
for asset_info in player.portfolio:
    asset = game_master.get_asset_by_id(asset_info["ID"])["instance"]
    print(f"資産名: {asset.name}, 帳簿価額: {int(asset.value)}, 累計減価償却額: {int(asset.accumulated_depreciation)}")

# 勘定元帳の記録を確認
print("\n=== プレイヤーの勘定元帳 ===")
player.ledger_manager.display_transaction_history()

In [ ]:
from asset import *
b = Building(name="Test Building", value=100000, owner=None, address="Test Address")
print(isinstance(b, Tangible))  # True を期待

In [ ]:
import asset
print(asset.__file__)


In [ ]:
from player import *

# ゲームマスターを初期化
game_master = GameMaster()

# プレイヤーを作成して登録
player = Player(name="Player1", game_master=game_master)
game_master.players.append(player)

# 資産の生成と登録
building_asset = game_master.construct_instance(
    asset_type="building",
    name="Office Building",
    value=1000000,
    address="Downtown City"
)

# プレイヤーに建物資産を取得させる
player.aquire_building(asset_id=building_asset["ID"], value=1000000)

# ゲーム内時間を進める
print("\n=== 90日進行 ===")
game_master.advance_time(days=90)

# 資産の確認
print("\n=== プレイヤーのポートフォリオ ===")
for asset_info in player.portfolio:
    asset = asset_info.get("instance")
    print(f"資産名: {asset.name}, 帳簿価額: {int(asset.value)}, 累計減価償却額: {int(asset.accumulated_depreciation)}")

# 勘定元帳の記録を確認
print("\n=== プレイヤーの勘定元帳 ===")
player.ledger_manager.display_transaction_history()

# 金利変動シミュレーションプログラム

## 概要
このプログラムは、景気シグナルに基づいて金利変動をシミュレーションし、時間経過に伴う累積金利の推移をプロットします。シミュレーションは複数回行われ、結果を比較可能な形で視覚化します。

## 特徴
- **景気シグナル**: 「良い」「やや良い」「中立」「やや悪い」「悪い」の5段階のシグナルを使用。
- **金利変動**: 金利の増減（0.5、0.25、0、-0.25、-0.5）を累積して追跡。
- **確率行列**:
  - 景気シグナル -> 金利変動の遷移確率。
  - 金利変動 -> 次期景気シグナルへの影響確率。
- **複数シミュレーション**: 各シミュレーションの結果をグラフで比較。

## 必要な環境
- Python 3.7以上
- 必要なライブラリ:
  - `numpy`
  - `random`
  - `matplotlib`

## 使用方法
1. 必要なライブラリをインストールしてください。
   ```bash
   pip install matplotlib numpy


In [ ]:
import numpy as np
import random
import matplotlib.pyplot as plt

# 遷移確率行列（景気シグナル -> 金利変動）
transition_matrix = {
    "良い":         [0.5, 0.5, 0, 0, 0],
    "やや良い":     [0.25, 0.25, 0.5, 0, 0],
    "中立":         [0, 0.25, 0.5, 0.25, 0],
    "やや悪い":     [0, 0, 0.5, 0.25, 0.25],
    "悪い":         [0, 0, 0, 0.5, 0.5]
}

# 金利推移と景気シグナルへの影響確率行列(金利変動 -> 次期の景気シグナル)
interest_to_signal_matrix = {
    0.5:            [0, 0, 0.17, 0.33, 0.5],
    0.25:           [0, 0.17, 0.33, 0.33, 0.17],
    0:              [0.2, 0.2, 0.2, 0.2, 0.2],
    -0.25:          [0.17, 0.33, 0.33, 0.17, 0],
    -0.5:           [0.5, 0.33, 0.17, 0, 0]
}

# 金利変動リスト
interest_rate_changes = [0.5, 0.25, 0, -0.25, -0.5]

# 景気シグナルのリスト
signals = ["良い", "やや良い", "中立", "やや悪い", "悪い"]

# シグナルから次期の金利変動をシミュレーションする関数
def simulate_next_rate(current_signal):
    if current_signal not in transition_matrix:
        raise ValueError("無効な景気シグナルです。")

    # 現在のシグナルに基づく遷移確率を取得
    probabilities = transition_matrix[current_signal]

    # 金利変動を確率的に選択
    next_rate = random.choices(interest_rate_changes, weights=probabilities, k=1)[0]
    return next_rate

# 金利変動から次期の景気シグナルをシミュレーションする関数
def simulate_next_signal(current_signal, rate_change):
    # 現在のシグナルのインデックスを取得
    current_index = signals.index(current_signal)

    # 景気シグナル変動インデックスの計算
    probabilities = interest_to_signal_matrix[rate_change]
    shift = random.choices(range(-2, 3), weights=probabilities, k=1)[0]
    new_index = current_index + shift

    # 境界条件を考慮した新しいインデックスの調整
    new_index = max(0, min(len(signals) - 1, new_index))

    return signals[new_index]

# 景気シグナルと金利変動の推移をシミュレーションする関数
def simulate_economy(initial_signal, steps):
    current_signal = initial_signal
    current_rate = 2  # 初期金利を0とする
    rate_history = []

    for step in range(steps):
        # 次期の金利変動をシミュレーション
        next_rate_change = simulate_next_rate(current_signal)

        # 累積金利を更新
        current_rate += next_rate_change

        # 次期の景気シグナルをシミュレーション
        next_signal = simulate_next_signal(current_signal, next_rate_change)

        rate_history.append((current_signal, current_rate, next_signal))

        # 現在の状態をプリント
        print(f"FY {step + 1}:(期首) 景気シグナル = {current_signal},(期末) 金利水準 = {current_rate}, 次の景気シグナル = {next_signal}")

        # 現在のシグナルを更新
        current_signal = next_signal

    return rate_history

# 初期設定
initial_signal = "中立"
steps = 10
simulations = 1

# シミュレーション実行
all_simulations = []
for _ in range(simulations):
    rate_history = simulate_economy(initial_signal, steps)
    all_simulations.append([rate for _, rate, _ in rate_history])

# グラフの描画
plt.figure(figsize=(10, 6))
for simulation in all_simulations:
    time_steps = list(range(1, steps + 1))
    plt.plot(time_steps, simulation, marker="o", linestyle="-", alpha=0.7)

plt.title("時間経過に伴う累積金利 (複数シミュレーション)")
plt.xlabel("時間 (ステップ)")
plt.ylabel("累積金利")
plt.grid(True)
plt.show()


